## Multi-Mission Grand Mesa Example

### Purpose
Demonstrate how to process and sample the various datasets SlideRule supports over the Grand Mesa Colorado region.

#### Import Packages

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import geopandas
import sliderule
from sliderule import icesat2, gedi, earthdata

#### Initialize SlideRule Python Client

In [ ]:
sliderule.init("slideruleearth.io", verbose=True)

#### Setup Processing Parameters
* Single granule over the Grand Mesa region of interest
* Run PhoREAL algorithm at 1m vertical bin resolution
* Sampling LandSat HLS data
* Sampling GEDI L4B data

In [ ]:
asset = "icesat2"
resource = "ATL03_20220105023009_02111406_005_01.h5"

In [ ]:
region = sliderule.toregion('grandmesa.geojson')
catalog = earthdata.stac(short_name="HLS", polygon=region["poly"], time_start="2022-01-01T00:00:00Z", time_end="2022-03-01T00:00:00Z", as_str=True)

In [ ]:
samples = {
    "landsat": {
        "asset": "landsat-hls",
        "catalog": catalog,
        "closest_time": "2022-01-05T00:00:00Z", 
        "bands": ["NDVI"]
    },
    "gedi": {
        "asset": "gedil4b"
    }   
}

In [ ]:
parms = { 
    "poly": region['poly'],
    "ats": 5.0,
    "cnt": 5,
    "len": 20.0,
    "res": 10.0,
    "atl08_class": [
        "atl08_ground", 
        "atl08_canopy", 
        "atl08_top_of_canopy"
    ],
    "maxi": 1,
    "phoreal": {
        "binsize": 1.0, 
        "geoloc": "center", 
        "use_abs_h": False, 
        "send_waveform": False
    },
    "samples": samples
}

#### Make ICESat-2 Processing Request to SlideRule (with LandSat and GEDI sampling)

In [ ]:
atl08 = icesat2.atl08p(parms, asset=asset, resources=[resource], keep_id=True)

#### Display ATL08 GeoDataFrame
Notice the columns that start with "landsat" and "gedi"

In [ ]:
atl08

In [ ]:
atl08.keys()

#### Print Out File Directory
When a GeoDataFrame includes samples from rasters, each sample value has a file id that is used to look up the file name of the source raster for that value.

In [ ]:
atl08.attrs['file_directory']

#### Make GEDI Process Request to SlideRule

In [ ]:
# Build GEDI L4A Request Parameters
parms = {
    "poly": region["poly"],
    "degrade_flag": 0,
    "l2_quality_flag": 1,
    "beam": 0,
    "samples": samples
}

# Turn verbose off
#sliderule.set_verbose(False)

# Request GEDI L4A Data
gedi04a = gedi.gedi04ap(parms)                      

#### Display GEDI 04A GeoDataFrame

In [ ]:
gedi04a

#### Plot GEDI L4A Data

In [ ]:
f, ax = plt.subplots(1, 1, figsize=[12,8])
ax.set_title("Above Ground BioDensity")
ax.set_aspect('equal')
vmin_hr, vmax_hr = gedi04a['agbd'].quantile((0.2, 0.8))
gedi04a.plot(ax=ax, column='agbd', cmap='inferno', s=1.0, vmin=vmin_hr, vmax=vmax_hr)

#### Perform Spatial Join (nearest) on GEDI L4A and ATL08/GEDI L4B/LandSat HLS Data

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    mmds = geopandas.sjoin_nearest(atl08, gedi04a, how='left', lsuffix="atl08", rsuffix="gedi04a")

In [ ]:
mmds

In [ ]:
mmds.keys()

#### Plot the Different GEDI/ATL08-PhoREAL/LandSat Values

In [ ]:
# Setup Plot
fig,ax = plt.subplots(num=None, figsize=(10, 8))
fig.set_facecolor('white')
fig.canvas.header_visible = False
ax.set_title("SlideRule vs. Landsat NDVI")
ax.set_xlabel('UTC')
ax.set_ylabel('height (m)')
legend_elements = []

# Filter DataFrame
df = mmds[(mmds['rgt'] == 211) & (mmds['gt'] == 30) & (mmds['cycle'] == 14)]
df = df[df["landsat.value_gedi04a"] < 100]
df = df[df["gedi.value_gedi04a"] > -100]

# Plot SlideRule ATL08 Vegetation Photon Counts
sc1 = ax.scatter(df.index.values, df["veg_ph_count"].values, c='red', s=2.5)
legend_elements.append(matplotlib.lines.Line2D([0], [0], color='red', lw=6, label='ATL08'))

# Plot GEDI L4B AGBD
sc2 = ax.scatter(df.index.values, df["gedi.value_gedi04a"].values, c='blue', s=2.5)
legend_elements.append(matplotlib.lines.Line2D([0], [0], color='blue', lw=6, label='L4B AGBD'))

# Plot GEDI L4A AGBD
sc3 = ax.scatter(df.index.values, df["agbd"].values, c='green', s=2.5)
legend_elements.append(matplotlib.lines.Line2D([0], [0], color='green', lw=6, label='L4A AGBD'))

# Plot LandSat NVDI
sc3 = ax.scatter(df.index.values, df["landsat.value_gedi04a"].values, c='orange', s=2.5)
legend_elements.append(matplotlib.lines.Line2D([0], [0], color='orange', lw=6, label='HLS NVDI'))


# Display Legend
lgd = ax.legend(handles=legend_elements, loc=2, frameon=True)
lgd.get_frame().set_alpha(1.0)
lgd.get_frame().set_edgecolor('white')

# Show Plot
plt.show()